In [1]:
!pip install datasets jiwer transformers sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.3 MB/s 
     |████████████████████████████████| 4.4 MB 48.6 MB/s 
     |████████████████████████████████| 1.2 MB 56.1 MB/s 
     |████████████████████████████████| 212 kB 45.7 MB/s 
     |████████████████████████████████| 1.1 MB 49.3 MB/s 
     |████████████████████████████████| 140 kB 50.8 MB/s 
     |████████████████████████████████| 101 kB 10.7 MB/s 
     |████████████████████████████████| 596 kB 50.3 MB/s 
     |████████████████████████████████| 127 kB 56.7 MB/s 
     |████████████████████████████████| 50 kB 4.5 MB/s 
     |████████████████████████████████| 6.6 MB 30.4 MB/s 
     |████████████████████████████████| 144 kB 42.2 MB/s 
     |████████████████████████████████| 271 kB 42.8 MB/s 
     |████████████████████████████████| 94 kB 3.5 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x8

## Download Dataset

In [2]:
from datasets import load_dataset
norm_data = load_dataset('VietAI/spoken_norm_assignment')
norm_data

Downloading:   0%|          | 0.00/1.01k [00:00<?, ?B/s]

Using custom data configuration VietAI--spoken_norm_assignment-d27a160dcd4fceae


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/VietAI___parquet/VietAI--spoken_norm_assignment-d27a160dcd4fceae/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['src', 'tgt'],
        num_rows: 500000
    })
    valid: Dataset({
        features: ['src', 'tgt'],
        num_rows: 2500
    })
    test: Dataset({
        features: ['src', 'tgt'],
        num_rows: 2500
    })
})

## Download Pretraided Language Model

In [3]:
from transformers.file_utils import cached_path, hf_bucket_url
from importlib.machinery import SourceFileLoader
import os
from transformers import EncoderDecoderModel

cache_dir = './cache'
model_name = 'nguyenvulebinh/envibert'

if not os.path.exists(cache_dir):
    os.makedirs(cache_dir)


def download_tokenizer_files():
    resources = ['envibert_tokenizer.py', 'dict.txt', 'sentencepiece.bpe.model']
    for item in resources:
        if not os.path.exists(os.path.join(cache_dir, item)):
            tmp_file = hf_bucket_url(model_name, filename=item)
            tmp_file = cached_path(tmp_file, cache_dir=cache_dir)
            os.rename(tmp_file, os.path.join(cache_dir, item))

def init_tokenizer():
    download_tokenizer_files()
    tokenizer = SourceFileLoader("envibert.tokenizer",
                                 os.path.join(cache_dir,
                                              'envibert_tokenizer.py')).load_module().RobertaTokenizer(cache_dir)
    return tokenizer

def init_model():
    download_tokenizer_files()
    tokenizer = SourceFileLoader("envibert.tokenizer",
                                 os.path.join(cache_dir,
                                              'envibert_tokenizer.py')).load_module().RobertaTokenizer(cache_dir)
    # set encoder decoder tying to True
    roberta_shared = EncoderDecoderModel.from_encoder_decoder_pretrained(model_name,
                                                                         model_name,
                                                                         tie_encoder_decoder=False)

    # set special tokens
    roberta_shared.config.decoder_start_token_id = tokenizer.bos_token_id
    roberta_shared.config.eos_token_id = tokenizer.eos_token_id
    roberta_shared.config.pad_token_id = tokenizer.pad_token_id

    # sensible parameters for beam search
    # set decoding params
    roberta_shared.config.max_length = 100
    roberta_shared.config.early_stopping = True
    roberta_shared.config.no_repeat_ngram_size = 3
    roberta_shared.config.length_penalty = 2.0
    roberta_shared.config.num_beams = 1
    roberta_shared.config.vocab_size = roberta_shared.config.encoder.vocab_size

    return roberta_shared, tokenizer

In [ ]:
model, tokenizer = init_model()
model

## Prepare Data Processing Function

In [5]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq, PreTrainedTokenizerBase
from dataclasses import dataclass
from transformers.utils import PaddingStrategy
from typing import Optional, Union, Any
import numpy as np
from datasets import load_metric
import torch

In [ ]:
# The DataCollator class is used for tokenization and padding to make batch input
@dataclass
class DataCollatorForEnViMT:
    tokenizer: PreTrainedTokenizerBase
    model: Optional[Any] = None
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    label_pad_token_id: int = -100
    return_tensors: str = "pt"

    def __call__(self, features, return_tensors=None):
        features_tokenized = []
        for feature in features:
          src = feature['input_ids']
          tgt = feature['labels']
          if len(src) == 0 or len(tgt) == 0:
              continue
          features_tokenized.append({'input_ids': tokenizer(src)["input_ids"], 
                                     'labels': tokenizer(tgt)["input_ids"][1:]})

        features = features_tokenized
        import numpy as np

        if return_tensors is None:
            return_tensors = self.return_tensors
        labels = [feature["labels"] for feature in features] if "labels" in features[0].keys() else None
        # We have to pad the labels before calling `tokenizer.pad` as this method won't pad them and needs them of the
        # same length to return tensors.
        if labels is not None:
            max_label_length = max(len(l) for l in labels)
            if self.pad_to_multiple_of is not None:
                max_label_length = (
                    (max_label_length + self.pad_to_multiple_of - 1)
                    // self.pad_to_multiple_of
                    * self.pad_to_multiple_of
                )

            padding_side = self.tokenizer.padding_side
            for feature in features:
                remainder = [self.label_pad_token_id] * (max_label_length - len(feature["labels"]))
                
                if isinstance(feature["labels"], list):
                    feature["labels"] = (
                        feature["labels"] + remainder if padding_side == "right" else remainder + feature["labels"]
                    )
                elif padding_side == "right":
                    feature["labels"] = np.concatenate([feature["labels"], remainder]).astype(np.int64)
                else:
                    feature["labels"] = np.concatenate([remainder, feature["labels"]]).astype(np.int64)
        
        features = self.tokenizer.pad(
            features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors=return_tensors,
        )

        # prepare decoder_input_ids
        if (
            labels is not None
            and self.model is not None
            and hasattr(self.model, "prepare_decoder_input_ids_from_labels")
        ):
            decoder_input_ids = self.model.prepare_decoder_input_ids_from_labels(labels=features["labels"])
            features["decoder_input_ids"] = decoder_input_ids

        return features

In [ ]:
# Test data collator
data_collator = DataCollatorForEnViMT(tokenizer, model=model)
data_collator([norm_data['train'][item] for item in range(2)])

## Training

In [ ]:
wer = load_metric('wer')

In [ ]:
# Define a function for evaluation
def get_metric_compute_fn(tokenizer):
    metric = wer

    def compute_metrics(eval_preds):
        preds, labels = eval_preds
        if isinstance(preds, tuple):
            preds = preds[0]

        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

        # Replace -100s in the labels as we can't decode them
        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Some simple post-processing
        decoded_preds = [pred.strip() for pred in decoded_preds]
        decoded_labels = [[label.strip()] for label in decoded_labels]

        result = metric.compute(predictions=decoded_preds, references=decoded_labels)
        return {"wer": result["score"]}

    return compute_metrics

In [ ]:
# instantiate training arguments
# TODO: fill in the arguments correctly
num_epochs = 10
checkpoint_path = "./envi_checkpoints"
batch_size = 16  # change to 16 for full training
training_args = Seq2SeqTrainingArguments(
    output_dir=checkpoint_path,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    gradient_accumulation_steps=1,
    predict_with_generate=True,
    save_total_limit=2,
    do_train=True,
    do_eval=True,
    logging_steps=10,
    num_train_epochs = num_epochs,
    warmup_ratio=1 / num_epochs,
    logging_dir=os.path.join(checkpoint_path, 'log'),
    overwrite_output_dir=True,
    metric_for_best_model='wer',
    greater_is_better=True,
    eval_accumulation_steps=10,
    dataloader_num_workers=20,
    # sharded_ddp="simple",
    # fp16=True,
)

In [ ]:
# instantiate trainer
# TODO: fill in the arguments correctly
# TODO: should train data be split for train_dataset and eval_dataset here? 
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    compute_metrics=get_metric_compute_fn(tokenizer),
    train_dataset=norm_data['train'], #.shard(200, 0),    # Only use subset of the dataset for a quick training. Remove shard for full training
    eval_dataset=norm_data['valid'], #.shard(100, 0), # Only use subset of the dataset for a quick training. Remove shard for full training
    data_collator=data_collator,
    tokenizer=tokenizer
)

In [ ]:
trainer.train()

## Load the trained Model 

In [ ]:
#TODO: insert the last checkpoint correctly
trained_model = model.from_pretrained('./envi_checkpoints/checkpoint-9020')

## Inference Results on Validation Data

In [ ]:
#TODO: Fill in the predictions and references
predictions = [' '.join(item) for item in norm_data['valid']['src']]
references = [' '.join(item) for item in norm_data['valid']['tgt']]
wer.compute(predictions=predictions,
            references=references)